# Construcción de la base de datos para clasificación

In [1]:
import numpy as np
import pandas as pd
import pickle
import scipy.stats as stats
import matplotlib.pyplot as plt

In [2]:
fecha_ref = pd.Timestamp('2025-01-01 00:00:00').floor('d')

In [3]:
with open("../procData/muestras_ovul_horas_norm1.pkl", "rb") as f:
    muestras_ovul = pickle.load(f)

### Filtrar series con alta desviación

In [ ]:
desviaciones = []

for key, datos in muestras_ovul.items():
    df = datos["serie"]
    std = df["result"].std()
    desviaciones.append((key, std))

# DataFrame para análisis
df_std = pd.DataFrame(desviaciones, columns=["serie_id", "std"])

# Umbrales usando IQR
q1 = df_std["std"].quantile(0.25)
q3 = df_std["std"].quantile(0.75)
iqr = q3 - q1
lim_sup = q3 + 1.5 * iqr

# Opción alternativa más restrictiva:
# lim_inf = df_std["std"].mean() - 2 * df_std["std"].std()
# lim_sup = df_std["std"].mean() + 2 * df_std["std"].std()

# Filtrar series dentro del rango
series_filtradas_std = df_std[(df_std["std"] <= lim_sup)]["serie_id"].tolist()

In [5]:
# Dataset limpio: solo series con desviación típica aceptable
muestras_ovul_filtrado = {}

for serie_id in series_filtradas_std:
    if serie_id in muestras_ovul:
        muestras_ovul_filtrado[serie_id] = muestras_ovul[serie_id]


print(f"Total original de series: {len(muestras_ovul)}")
print(f"Total después del filtrado por desviación típica: {len(muestras_ovul_filtrado)}")

Total original de series: 59
Total después del filtrado por desviación típica: 58


### Series positivas

In [6]:
def filtrar_por_horas(df, hora_inicio, hora_fin):
    horas = df["resultTimestamp"].dt.hour
    if hora_inicio <= hora_fin:
        # Caso simple: por ejemplo 6 a 12
        return df[(horas >= hora_inicio) & (horas <= hora_fin)]
    else:
        # Caso cruzando medianoche: por ejemplo 21 a 3
        return df[(horas >= hora_inicio) | (horas <= hora_fin)]


In [7]:
intervalo_dias = [-3, 4] # Queremos incluir ambos extremos del intervalo
intervalo_horas = [21, 3] # Queremos incluir ambos extremos del intervalo

if intervalo_horas[0] <= intervalo_horas[1]:
    num_horas_intervalo = intervalo_horas[1] - intervalo_horas[0] + 1
else:
    num_horas_intervalo = (24 - intervalo_horas[0]) + (intervalo_horas[1] + 1)

num_horas = (intervalo_dias[1] - intervalo_dias[0] + 1) * num_horas_intervalo + 1
num_horas

57

In [19]:
num_horas_intervalo

7

In [8]:
muestras_positivas = {}

for serie_id, datos in muestras_ovul_filtrado.items():
    df = datos["serie"].copy()
    
    # Filtro por días relativos
    tiempo_rel = df["resultTimestamp"] - fecha_ref
    filtro_dias = (tiempo_rel >= pd.Timedelta(days=intervalo_dias[0])) & \
                  (tiempo_rel <= pd.Timedelta(days=intervalo_dias[1] + 1))
    df_filtrada = df[filtro_dias].copy()
    
    # Filtro por horas
    df_filtrada = filtrar_por_horas(df_filtrada, hora_inicio=intervalo_horas[0], hora_fin=intervalo_horas[1])

    # Guardar si hay datos suficientes
    if not df_filtrada.empty:
        muestras_positivas[serie_id] = {
            "serie": df_filtrada.reset_index(drop=True),
            "ovul": fecha_ref
        }

In [9]:
print(len(muestras_positivas))

longitudes = [len(datos["serie"]) for datos in muestras_positivas.values()]
all(l == num_horas for l in longitudes)

58


True

In [10]:
X_pos = []

for serie_id, datos in muestras_positivas.items():
    df = datos["serie"].copy()
    df = df.sort_values("resultTimestamp")  # Asegurar orden temporal
    
    valores = df["result"].values

    X_pos.append(valores)


columnas = [f"t_{i}" for i in range(0, num_horas)] 

df_series_positivas = pd.DataFrame(X_pos, columns=columnas)

### Series negativas

In [11]:
muestras_negativas_antes = {}

for serie_id, datos in muestras_ovul_filtrado.items():
    df = datos["serie"].copy()
    
    tiempo_rel = df["resultTimestamp"] - fecha_ref
    filtro = (tiempo_rel >= pd.Timedelta(days=-9)) & (tiempo_rel <= pd.Timedelta(days=-1))
    
    df_filtrada = df[filtro].copy().reset_index(drop=True)

    df_filtrada = filtrar_por_horas(df_filtrada, hora_inicio=intervalo_horas[0], hora_fin=intervalo_horas[1])

    muestras_negativas_antes[serie_id] = {
                "serie": df_filtrada,
                "ovul": fecha_ref
            }

In [12]:
muestras_negativas_despues = {}

for serie_id, datos in muestras_ovul_filtrado.items():
    df = datos["serie"].copy()
    
    tiempo_rel = df["resultTimestamp"] - fecha_ref
    filtro = (tiempo_rel >= pd.Timedelta(days=2)) & (tiempo_rel <= pd.Timedelta(days=10))
    
    df_filtrada = df[filtro].copy().reset_index(drop=True)

    df_filtrada = filtrar_por_horas(df_filtrada, hora_inicio=intervalo_horas[0], hora_fin=intervalo_horas[1])

    muestras_negativas_despues[serie_id] = {
                "serie": df_filtrada,
                "ovul": fecha_ref
            }

In [13]:
print(len(muestras_negativas_antes))
print(len(muestras_negativas_despues))

longitudes_antes = [len(datos["serie"]) for datos in muestras_negativas_antes.values()]
print(all(l == num_horas for l in longitudes_antes))

longitudes_despues = [len(datos["serie"]) for datos in muestras_negativas_despues.values()]
print(all(l == num_horas for l in longitudes_despues))

58
58
True
True


In [14]:
X_neg = []

for serie_id, datos in muestras_negativas_antes.items():
    df = datos["serie"].copy()
    df = df.sort_values("resultTimestamp")  # Asegurar orden temporal
    valores = df["result"].values
    X_neg.append(valores)

for serie_id, datos in muestras_negativas_despues.items():
    df = datos["serie"].copy()
    df = df.sort_values("resultTimestamp")  # Asegurar orden temporal
    valores = df["result"].values
    X_neg.append(valores)


columnas = [f"t_{i}" for i in range(0, num_horas)] 

df_series_negativas = pd.DataFrame(X_neg, columns=columnas)

In [15]:
# Etiquetar los conjuntos
df_series_positivas["label"] = 1
df_series_negativas["label"] = 0

# Unificar
df_completo = pd.concat([df_series_positivas, df_series_negativas], ignore_index=True)

# Mostrar resumen
num_positivas = df_series_positivas.shape[0]
num_negativas = df_series_negativas.shape[0]

print(f"Número de muestras positivas: {num_positivas}")
print(f"Número de muestras negativas: {num_negativas}")
print(f"Total de muestras: {df_completo.shape[0]}")

Número de muestras positivas: 58
Número de muestras negativas: 116
Total de muestras: 174


In [16]:
df_completo

,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,t_9,...,t_48,t_49,t_50,t_51,t_52,t_53,t_54,t_55,t_56,label
0,36.369506,36.445393,36.476059,36.579726,36.495129,36.510726,36.550393,36.567393,36.530059,36.556726,...,36.567735,36.470594,36.363055,36.478064,36.606093,36.513609,36.552768,36.536643,36.502069,1
1,36.329549,36.363716,36.363716,36.363716,36.363716,36.363716,36.363716,36.363716,36.363716,36.363716,...,36.910982,36.888416,36.772277,36.748570,36.878558,36.710686,36.820710,36.615203,36.577215,1
2,36.529684,36.505774,36.517708,36.413560,36.356226,36.264115,36.284450,36.178043,35.995197,35.956195,...,36.481226,36.436893,36.453185,36.433585,36.423226,36.423011,36.426724,36.339037,36.372914,1
3,36.369121,36.476743,36.533076,36.489076,36.518516,36.518516,36.518516,36.518516,36.518516,36.518516,...,36.500410,36.427076,36.461410,36.311729,36.321020,36.550743,36.638943,36.692510,36.570243,1
4,36.540847,36.556513,36.494513,36.484180,36.640513,36.569513,36.503524,36.509206,36.507408,36.526513,...,36.641833,36.569513,36.520180,36.592847,36.574862,36.609180,36.550027,36.580847,36.550180,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,36.664017,36.535351,36.351017,36.445684,36.580139,36.718351,36.861684,36.740827,36.660261,36.781684,...,36.754042,36.684772,36.504452,36.671579,36.671358,36.687695,36.868684,36.957767,36.487017,0
170,36.735193,36.755460,36.621460,36.661794,36.654460,36.703794,36.496794,36.514460,36.693794,36.680460,...,36.494127,36.494127,36.494127,36.494127,36.494127,36.494127,36.494127,36.494127,36.494127,0
171,36.546749,36.640105,36.592957,36.384360,36.658105,36.560438,36.639105,36.656105,36.636105,36.613105,...,36.573991,36.574105,36.601105,36.641438,36.665438,36.686105,36.640105,36.596438,36.514833,0
172,36.674175,36.614324,36.681703,36.704525,36.840268,36.770990,36.645036,36.671370,36.689036,36.591703,...,36.857411,36.859319,36.942658,37.070345,36.977359,36.545703,36.545703,36.545703,36.545703,0


In [17]:
df_completo.to_csv("../procData/dataset_series_temporales_tramos.csv", index=False)

In [18]:
df_series_positivas.to_csv("../procData/dataset_series_temporales_tramos_pos.csv", index=False)